In [48]:
from collections import defaultdict
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

DIAS_MES_PADRAO = 30

def vfuturo(vp, i, n):
    return vp*(1+i)**n

def vpresente(vf, i, n):
    return vf/(1+i)**n

def taxa_mensal_para_dia(i):
    return (1 + i) ** (1/DIAS_MES_PADRAO) - 1

# Relação prestação x valor solicitado

$$ PV = PMT * FPV(i,n) $$

com:
 
$$ FPV(i, n) = \frac{1 - (1 + i)^{-n}}{i}$$

Onde:

- PV: Valor solicitado
- PMT: Valor da prestação
- n: Número de prestações
- i: taxa de jutos


## Informando o valor solicitado 

In [49]:
taxa_mensal = 0.02
valor_solicitado = 8_000
numero_de_prestacoes = 24
data_liberacao = date(2021, 1, 15)
dia_vencimento_parcela = 15
meses_de_carencia = 0

In [50]:
taxa_dia = taxa_mensal_para_dia(taxa_mensal)
taxa_dia

0.0006603054822866827

In [51]:
data_vecimento_carencia = [
    (data_liberacao + relativedelta(months=prestacao)).replace(day=dia_vencimento_parcela)
    for prestacao in range(1, meses_de_carencia + 1)
]

data_vencimento_parcelas = [
    (data_liberacao + relativedelta(months=prestacao)).replace(day=dia_vencimento_parcela)
    for prestacao in range(meses_de_carencia + 1, numero_de_prestacoes+meses_de_carencia + 1)
]


datas_vencimento = data_vecimento_carencia + data_vencimento_parcelas
pd.DataFrame({"mes": range(1, numero_de_prestacoes + meses_de_carencia + 1), "datas": datas_vencimento})

,mes,datas
0,1,2021-02-15
1,2,2021-03-15
2,3,2021-04-15
3,4,2021-05-15
4,5,2021-06-15
5,6,2021-07-15
6,7,2021-08-15
7,8,2021-09-15
8,9,2021-10-15
9,10,2021-11-15


In [52]:
total_dias = (datas_vencimento[-1] - data_liberacao).days
total_dias
print(f"{total_dias=}")

total_dias=730


In [ ]:
saldo_devedor = valor_solicitado

tabela = defaultdict(list)

tabela['mes'].append(0)
tabela['saldo_devedor'].append(saldo_devedor)
tabela['amortizacao'].append(0)
tabela['juros'].append(0)
tabela['valor_prestacao'].append(0)
tabela['valor_presente_prestacao'].append(0)
tabela['dias_do_mes'].append(0)
tabela['dias_acumulados'].append(0)
tabela['data_vencimento'].append(0)
tabela['tipo'].append('Inicial')
tabela['juros_formula'].append(0)
tabela['total_dias'].append(0)

for mes in range(1, meses_de_carencia + 1):

    dia_vencimento_atual = datas_vencimento[mes - 1]
    if mes == 1:
        dias_do_mes = (dia_vencimento_atual - data_liberacao).days
    else:
        dia_vencimento_anterior = datas_vencimento[mes - 2]
        dias_do_mes = (dia_vencimento_atual - dia_vencimento_anterior).days
    dias_acumulados = (dia_vencimento_atual - data_liberacao).days

    juros = vfuturo(saldo_devedor, taxa_dia, dias_do_mes) - saldo_devedor

    saldo_devedor *= (1+taxa_mensal)
    tabela['mes'].append(mes + 1)
    tabela['saldo_devedor'].append(saldo_devedor)
    tabela['amortizacao'].append(0)
    tabela['juros'].append(juros)
    tabela['valor_prestacao'].append(0)
    tabela['valor_presente_prestacao'].append(0)
    tabela['dias_do_mes'].append(dias_do_mes)
    tabela['dias_acumulados'].append(dias_acumulados)
    tabela['data_vencimento'].append(dia_vencimento_atual)
    tabela['tipo'].append('carencia')
    tabela['juros_formula'].append(0)


amortizacao = saldo_devedor / numero_de_prestacoes
print(f"{amortizacao=:10.2f}")

for mes in range(meses_de_carencia + 1, meses_de_carencia + numero_de_prestacoes+1):

    dia_vencimento_atual = datas_vencimento[mes - 1]
    if mes == 1:
        dias_do_mes = (dia_vencimento_atual - data_liberacao).days
    else:
        dia_vencimento_anterior = datas_vencimento[mes - 2]
        dias_do_mes = (dia_vencimento_atual - dia_vencimento_anterior).days
    dias_acumulados = (dia_vencimento_atual - data_liberacao).days

    juros = vfuturo(saldo_devedor, taxa_dia, dias_do_mes) - saldo_devedor
    valor_da_prestacao = amortizacao + juros
    saldo_devedor -= amortizacao

    tabela['mes'].append(mes)
    tabela['data_vencimento'].append(dia_vencimento_atual)
    tabela['dias_do_mes'].append(dias_do_mes)
    tabela['dias_acumulados'].append(dias_acumulados)
    tabela['saldo_devedor'].append(saldo_devedor)
    tabela['amortizacao'].append(amortizacao)
    tabela['juros'].append(juros)
    tabela['valor_prestacao'].append(valor_da_prestacao)
    tabela['valor_presente_prestacao'].append(vpresente(valor_da_prestacao, taxa_dia, dias_acumulados))
    tabela['tipo'].append('parcela')

    juros_formula = amortizacao * (numero_de_prestacoes - mes + 1) * ((1 + taxa_dia) ** dias_do_mes - 1)
    tabela['juros_formula'].append(juros_formula)
    tabela['total_dias'].append(total_dias)


amortizacao=    333.33
165.38809273546383 8000.0 1.020673511591933
143.01637523055442 7666.666666666666 1.018654309812681
151.6057516741752 7333.333333333333 1.020673511591933
140.00000000000477 7000.0 1.0200000000000007
137.82341061288653 6666.666666666666 1.020673511591933
126.66666666667099 6333.333333333333 1.0200000000000007
124.04106955159789 6000.0 1.020673511591933
117.14989902095354 5666.666666666666 1.020673511591933
106.66666666667031 5333.333333333333 1.0200000000000007
103.36755795966491 5000.0 1.020673511591933
93.33333333333651 4666.666666666666 1.0200000000000007
89.58521689837625 4333.333333333333 1.020673511591933
82.69404636773191 4000.0 1.020673511591933
68.39913597983038 3666.6666666666665 1.018654309812681
68.91170530644327 3333.333333333333 1.020673511591933
60.00000000000205 3000.0 1.0200000000000007
55.12936424515461 2666.6666666666665 1.020673511591933
46.666666666668256 2333.333333333333 1.0200000000000007
41.34702318386596 2000.0 1.020673511591933
34.4558526

In [56]:
tabela_valor_solicidado_dado = pd.DataFrame(tabela)
tabela_valor_solicidado_dado.style.format({
    'dias_parcela': '{:d}',
    'dias_acumulados': '{:d}',
    'saldo_devedor': '{:.2f}',
    'amortizacao': '{:.2f}',
    'juros': '{:.2f}',
    'valor_prestacao': '{:.2f}',
    'valor_presente_prestacao': '{:.2f}',
    'juros_formula': '{:.2f}',
})

,mes,saldo_devedor,amortizacao,juros,valor_prestacao,valor_presente_prestacao,dias_do_mes,dias_acumulados,data_vencimento,tipo,juros_formula,total_dias
0,0,8000.00,0.00,0.00,0.00,0.00,0,0,0,Inicial,0.00,0
1,1,7666.67,333.33,165.39,498.72,488.62,31,31,2021-02-15,parcela,165.39,730
2,2,7333.33,333.33,143.02,476.35,458.15,28,59,2021-03-15,parcela,143.02,730
3,3,7000.00,333.33,151.61,484.94,456.97,31,90,2021-04-15,parcela,151.61,730
4,4,6666.67,333.33,140.00,473.33,437.29,30,120,2021-05-15,parcela,140.00,730
5,5,6333.33,333.33,137.82,471.16,426.46,31,151,2021-06-15,parcela,137.82,730
6,6,6000.00,333.33,126.67,460.00,408.20,30,181,2021-07-15,parcela,126.67,730
7,7,5666.67,333.33,124.04,457.37,397.65,31,212,2021-08-15,parcela,124.04,730
8,8,5333.33,333.33,117.15,450.48,383.72,31,243,2021-09-15,parcela,117.15,730
9,9,5000.00,333.33,106.67,440.00,367.44,30,273,2021-10-15,parcela,106.67,730


In [55]:
tabela_valor_solicidado_dado[['amortizacao', 'juros', 'valor_prestacao', 'valor_presente_prestacao']].sum()

amortizacao                  8000.000000
juros                        2023.812517
valor_prestacao             10023.812517
valor_presente_prestacao     8000.000000
dtype: float64